<a href="https://colab.research.google.com/github/duper203/upstage_cookbook/blob/main/DocVision_3_Generate_webpage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Upstage DocVision#3] Build a simple Webpage from your drawing



This will walk through an example of how to use the Upstage Solar DocVision API with LangChain to generate a simple webpage. The result will allow users to select their favorite ice cream flavor, based on the provided image.

# STEP 1: Requirements

In [ ]:
pip install langchain-upstage

In [14]:
from google.colab import userdata
upstage_api= userdata.get('upstage_api_key')

# STEP 2: Encoding an Image
To process the image with DocVision, you first need to encode it in base64 format. Whether the image is local or from a URL, the following function will help encode it:



In [19]:
import base64
import requests

def get_base64_encoded_image(image_source):
    if image_source.startswith("http://") or image_source.startswith("https://"):
        response = requests.get(image_source)
        binary_data = response.content
    else:
        with open(image_source, "rb") as image_file:
            binary_data = image_file.read()

    base_64_encoded_data = base64.b64encode(binary_data)
    base64_string = base_64_encoded_data.decode("utf-8")
    return base64_string

# Step 3: Generating an HTML Webpage with ChatUpstage


In [ ]:
from langchain_upstage import ChatUpstage
from langchain_core.messages import HumanMessage

chat = ChatUpstage(api_key=upstage_api, model="solar-docvision")

file_path = 'website2.png'

messages = [
    HumanMessage(
        content=[
            {"type": "text",
            "text": f"Using the image as a reference for layout and design, create a responsive HTML webpage that allows users to choose their favorite ice cream flavor. The page should include a form with dropdown or radio button options for different flavors, a submit button, and a visually appealing design inspired by the layout of the image"},
            {"type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{get_base64_encoded_image(file_path)}"
                },
            },
        ],
    ),
]
response = chat.invoke(messages)
print(response.content)

output_filename = "webpage.html"
html = response.content
with open(output_filename, "w") as f:
    f.write(html)


## Display HTML

In [66]:
from IPython.display import HTML

display(HTML(html))

# STEP 4: Make it fancier~ with Solar Pro

In [74]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

chat_html = ChatUpstage(api_key=upstage_api, model="solar-pro")

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            """Without changing the basic html layout, design a visually appealing webpage with a light purple color scheme Feel free to add your own creative elements. only return the final HTML code.

            ---
            **the basic html:**
            {html}
            ---
            """,
        )
    ]
)

chain_pro = prompt_template | chat_html | StrOutputParser()
response = chain_pro.invoke(
            {
                "html": html,
            }
)
# print(response)

In [75]:
display(HTML(response))